In [5]:
import pandas as pd
import numpy as np 
import pickle

In [6]:
import pandas as pd

print(pd.__version__)


1.5.2


In [7]:
#gdis disaster info 
gdis = pd.read_csv('../../data/pend-gdis-1960-2018-disasterlocations.csv')

print(gdis.head(2))
#get emdat dataset
emdat = pd.read_csv('../../data/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])
emdat.head()

    id  country iso3   gwno  year  geo_id  geolocation  level     adm1  \
0  109  Albania  ALB  339.0  2009     346  Ana E Malit      3  Shkoder   
1  109  Albania  ALB  339.0  2009     351       Bushat      3  Shkoder   

       adm2         adm3     location  historical hist_country disastertype  \
0  Shkodres  Ana E Malit  Ana E Malit           0          NaN        flood   
1  Shkodres       Bushat       Bushat           0          NaN        flood   

  disasterno   latitude  longitude  
0  2009-0631  42.020948  19.418317  
1  2009-0631  41.959294  19.514309  


<ipython-input-7-7e3153f801b8>:6: DtypeWarning: Columns (8,16,17,18,19,24,25,26,27,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  emdat = pd.read_csv('../../data/emdat_public_2022_09_21_query_uid-47Yzpr.csv', skiprows=[0,1,2,3,4,5])


,Dis No,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations
0,1900-9002-CPV,1900,9002,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
1,1900-9001-IND,1900,9001,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.077091,NaN,NaN,NaN,NaN
2,1902-0012-GTM,1902,12,NaN,Natural,Geophysical,Earthquake,Ground movement,NaN,NaN,...,NaN,NaN,NaN,25000.0,781207.0,3.200175,NaN,NaN,NaN,NaN
3,1902-0003-GTM,1902,3,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN
4,1902-0010-GTM,1902,10,NaN,Natural,Geophysical,Volcanic activity,Ash fall,NaN,Santa Maria,...,NaN,NaN,NaN,NaN,NaN,3.200175,NaN,NaN,NaN,NaN


In [8]:
#select certain columns from emdat and join with gdis 
emdat['disasterno'] = emdat['Dis No'].str[:-4] #format disasterno to merge  
cols = ['disasterno', 'Year', 'Event Name', 
#         'Disaster Type', 'Disaster Subtype', 
#         'Region', 'Continent', #'Location',
        'Start Year', 'Start Month', 'Start Day', 
        'End Year', 'End Month','End Day',  
        "Total Damages, Adjusted ('000 US$)"] 

emdat = emdat[cols]


In [9]:
#join emdat and gdis 
gdis = pd.merge(emdat, gdis, on = 'disasterno', how='right')

In [10]:
# #format date time - it's not done because of missing values 
# df['year'] = df['Start Year']
# df['month'] = df['Start Month']
# df['day'] = df['Start Day']


# # 
# dff = df[['year','month','day']].fillna(0).astype(int)

# pd.to_datetime(dff[['year','month','day']])

In [11]:
#print 
gdis = gdis.drop_duplicates(subset=['id'])
gdis.shape

(9924, 27)

In [13]:
#new grid_id: round to integers 
gdis['lat_grid'] = gdis['latitude'].round().astype(int)
gdis['lon_grid'] = gdis['longitude'].round().astype(int)
gdis['grid_id'] = list(zip(gdis['lat_grid'],gdis['lon_grid'])) 
print('total number of grid pairs', len(gdis.grid_id.value_counts()))

# check if they lie in range
print('lon range', gdis['lon_grid'].min(), gdis['lon_grid'].max()) 
print('lat range', gdis['lat_grid'].min(), gdis['lat_grid'].max()) 

total number of grid pairs 2827
lon range -178 180
lat range -54 68


In [14]:
# #latitude range from -90 to 90, longitude range from -180 to 180 

# #convert lat and long into 1 degree grid: correspond to 100*100km 
# gdis['lat_grid'] = np.digitize(np.array(gdis['latitude']),np.arange(-90,90,1))
# gdis['lon_grid'] = np.digitize(np.array(gdis['longitude']),np.arange(-180,180,1)) 
# #compute the grid pair id 
# gdis['grid_id'] = list(zip(gdis['lat_grid'],gdis['lon_grid']))
# print('total number of grid pairs', len(gdis.grid_id.value_counts()))

In [15]:
gdis

,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,adm3,location,historical,hist_country,disastertype,latitude,longitude,lat_grid,lon_grid,grid_id
0,2009-0631,2009.0,NaN,2009.0,12.0,27.0,2010.0,1.0,8.0,NaN,...,Ana E Malit,Ana E Malit,0,NaN,flood,42.020948,19.418317,42,19,"(42, 19)"
2,2001-0146,2001.0,NaN,2001.0,4.0,2.0,2001.0,4.0,9.0,NaN,...,Onjiva,Onjiva,0,NaN,flood,-17.093484,15.665758,-17,16,"(-17, 16)"
3,2009-0092,2009.0,NaN,2009.0,3.0,1.0,2009.0,4.0,16.0,NaN,...,Evale,Evale,0,NaN,flood,-16.531533,15.773987,-17,16,"(-17, 16)"
18,2010-0105,2010.0,NaN,2010.0,3.0,1.0,2010.0,3.0,17.0,NaN,...,Evale,Evale,0,NaN,flood,-16.531533,15.773987,-17,16,"(-17, 16)"
20,1995-0082,1995.0,NaN,1995.0,5.0,15.0,1995.0,5.0,15.0,1422569.0,...,Hatiya,Hatiya,0,NaN,storm,22.291591,91.065456,22,91,"(22, 91)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82858,1960-0011,1960.0,Manam,1960.0,3.0,17.0,1960.0,3.0,17.0,NaN,...,NaN,Manam,1,Australia,volcanic activity,-4.383452,144.922859,-4,145,"(-4, 145)"
82859,2009-9633,2009.0,NaN,2009.0,NaN,NaN,2010.0,NaN,NaN,NaN,...,NaN,Unity,1,Sudan,drought,8.926421,29.885783,9,30,"(9, 30)"
82865,1990-9289,1990.0,NaN,1990.0,NaN,NaN,1990.0,NaN,NaN,NaN,...,NaN,Bar El Ghazal Province,1,Sudan,drought,8.248650,26.099769,8,26,"(8, 26)"
82866,1969-9069,1969.0,NaN,1969.0,NaN,NaN,1971.0,NaN,NaN,73867.0,...,NaN,Taiz-Torba,1,Yemen North,drought,13.418716,43.777557,13,44,"(13, 44)"


In [16]:
#create labels for NLP training purposes, count number of natural disasters for each location 
#filter first to flood 
gdis.groupby('grid_id')

In [9]:
# #count number of locations in each grid_id: ideally just 1 location per grid_id, if not, I can make grid finer. 
# gdis.groupby('grid_id').agg({'location':'nunique'}).sort_values(by='location').value_counts()

In [10]:
# # the grid with most regions 
# gdis.loc[gdis['grid_id']==(113, 295)].head(5)

# Add text data from Wikipedia

In [20]:
import wikipedia
#doc: https://pypi.org/project/wikipedia/ 

In [21]:
# #set up wiki api 
# wiki_wiki = wikipediaapi.Wikipedia('en')

# location = 'Anhui'
# page_py = wiki_wiki.page('Anhui')
# section_py = page_py.section_by_title('Geography')
# txt= section_py.text


In [25]:
# page_py.sections
wikipedia.summary("wikipedia")

"The Wikimedia Foundation, Inc., or Wikimedia for short and abbreviated as WMF, is an American 501(c)(3) nonprofit organization headquartered in San Francisco, California and registered as a charitable foundation under local laws. Best known as the hosting platform for Wikipedia, a crowdsourced online encyclopedia, it also hosts other related projects and MediaWiki, a wiki software.The Wikimedia Foundation was established in 2003 in St. Petersburg, Florida, by Jimmy Wales as a nonprofit way to fund Wikipedia, Wiktionary, and other crowdsourced wiki projects that had until then been hosted by Bomis, Wales's for-profit company. The Foundation finances itself mainly through millions of small donations from Wikipedia readers, collected through email campaigns and annual fundraising banners placed on Wikipedia and its sister projects. These are complemented by grants from philanthropic organizations and tech companies, and starting in 2022, by services income from Wikimedia Enterprise.\nThe

In [26]:
#count number of locations in each grid_id 
gdis.groupby('grid_id').agg({'location':'nunique'}).sort_values(by='location')

,location
grid_id,
"(-54, -72)",1
"(26, 63)",1
"(26, 60)",1
"(26, 33)",1
"(26, -78)",1
...,...
"(25, 101)",23
"(31, 103)",24
"(15, 121)",28


In [27]:
# the grid with most regions 
gdis.loc[gdis['grid_id']==(22, 114)].head(5)

,disasterno,Year,Event Name,Start Year,Start Month,Start Day,End Year,End Month,End Day,"Total Damages, Adjusted ('000 US$)",...,adm3,location,historical,hist_country,disastertype,latitude,longitude,lat_grid,lon_grid,grid_id
49079,1960-0042,1960.0,Mary,1960.0,6.0,4.0,1960.0,6.0,4.0,NaN,...,NaN,Cheung Chau,0,NaN,storm,22.251042,113.965900,22,114,"(22, 114)"
49088,1962-0015,1962.0,Wanda,1962.0,8.0,28.0,1962.0,8.0,28.0,NaN,...,NaN,Tai Po,0,NaN,storm,22.453359,114.239223,22,114,"(22, 114)"
49092,1968-0067,1968.0,Shirley,1968.0,8.0,21.0,1968.0,8.0,21.0,NaN,...,NaN,Kwangtung,0,NaN,storm,22.315088,114.226257,22,114,"(22, 114)"
49098,1970-0097,1970.0,NaN,1970.0,5.0,13.0,1970.0,5.0,13.0,NaN,...,NaN,Sha Tin area,0,NaN,flood,22.386781,114.199021,22,114,"(22, 114)"
49099,1970-0099,1970.0,NaN,1970.0,9.0,16.0,1970.0,9.0,16.0,NaN,...,NaN,Taipo area,0,NaN,flood,22.453359,114.239223,22,114,"(22, 114)"


In [28]:
#for each grid, get the most common location 
#process grid_id and location pairs 
grid_loc = gdis[['grid_id','location']]
#keep the first location in each grid 
# grid_loc = grid_loc.drop_duplicates(subset=['grid_id'], keep='first')
#aggregate the location informatin to a list 
grid_loc = grid_loc.groupby('grid_id')['location'].agg(list).reset_index()
# grid_loc['location'] = grid_loc['location'].str.split(',').str[0]

In [17]:
#TO DO: deal with duplicated locations -> use the list to attach txt, keep if geography is present. 


In [29]:
grid_loc

,grid_id,location
0,"(-54, -72)",[Punta Arenas]
1,"(-49, -73)",[ O'Higgins]
2,"(-49, -70)",[Santa Cruz]
3,"(-46, -73)","[Aysen region, Aisen del Gral. Carlos Ibañez d..."
4,"(-46, 168)","[Invercargill, Mataura Borough, Invercargil, S..."
...,...,...
2822,"(65, 37)",[Eastern Maritime region]
2823,"(66, -23)","[Sudavik, Flateyri]"
2824,"(66, 130)","[Sakha-Yakutia, Yakutia, Sakha Rep. provinces]"
2825,"(68, -135)",[Aklavik]


In [30]:
#attach wikipedia texts 


location = 'Anhui'
def get_wiki(location):
#initiaise txt with empty string 
    txt= np.nan 
    #search similar ords in wikipedia 
    similar_words = wikipedia.search(location)   
    for similar in similar_words:
        #try getting geographical information 
        try:
            txt_geo = wikipedia.WikipediaPage(similar).section('Geography')
        except:
            txt_geo = np.nan
        #
        if txt_geo != np.nan:
            txt = txt_geo
            break 
    return txt 

get_wiki(location)


'Anhui is topographically diverse. The north is part of the North China Plain while the north-central areas are part of the Huai River watershed. Both regions are flat, and densely populated. The land becomes more uneven further south, with the Dabie Mountains occupying much of southwestern Anhui and a series of hills and ranges cutting through southeastern Anhui, between which is the Yangtze River. The highest peak in Anhui is Lotus Peak, part of Huangshan in southeastern Anhui. It has an altitude of 1873 m.\nMajor rivers include the Huai River in the north and the Yangtze in the south. The largest lake is Lake Chaohu situated in the center of the province, with an area of about 800 km2 (310 sq mi). The southeastern part of the province near the Yangtze River has many lakes as well.\nAs with topography, the province differs in climate from north to south. The north is more temperate with more distinct seasons. January temperatures average at around −1 to 2 °C north of the Huai River, 

In [31]:
grid_loc

,grid_id,location
0,"(-54, -72)",[Punta Arenas]
1,"(-49, -73)",[ O'Higgins]
2,"(-49, -70)",[Santa Cruz]
3,"(-46, -73)","[Aysen region, Aisen del Gral. Carlos Ibañez d..."
4,"(-46, 168)","[Invercargill, Mataura Borough, Invercargil, S..."
...,...,...
2822,"(65, 37)",[Eastern Maritime region]
2823,"(66, -23)","[Sudavik, Flateyri]"
2824,"(66, 130)","[Sakha-Yakutia, Yakutia, Sakha Rep. provinces]"
2825,"(68, -135)",[Aklavik]


In [33]:
location_ls

grid_id                                            (-46, 168)
location    [Invercargill, Mataura Borough, Invercargil, S...
Name: 4, dtype: object

In [39]:
#search similar words in wikipedia 
location_ls = grid_loc.iloc[4]
location_ls['location'][1]
similar_words = []
for location in location_ls:
    print(location)
    similar_words.append(wikipedia.search(location))

    similar_words

(-46, 168)
[['168', '168 Sibylla', 'M168', 'UFC 168', '168 (number)', 'Tracy 168', 'VA168', 'Motel 168', 'California State Route 168', 'Class 168']]
['Invercargill', 'Mataura Borough', 'Invercargil', 'Southland county']
[['168', '168 Sibylla', 'M168', 'UFC 168', '168 (number)', 'Tracy 168', 'VA168', 'Motel 168', 'California State Route 168', 'Class 168'], ['Southland County', 'Southland', 'Southland District', 'List of Southland episodes', 'Southland, Texas', 'Southland Publishing', 'Southland District Council', 'Southland Academy', 'Southland Credit Union', 'Chicago Southland']]
Invercargill is the southernmost city in the Commonwealth of Nations. Invercargill is situated on the fertile and alluvial Southland Plains, which is amongst some of New Zealand's most fertile farmland. Southern Invercargill lies on the shore of the New River Estuary, while the northern parts lie on the banks of the Waihopai River. A tributary of the Waihopai is the Otepuni Stream or Creek, which flows from ea

WikipediaException: An unknown error occured: "Search request is longer than the maximum allowed length. (Actual: 1274; allowed: 300)". Please report it on GitHub!

In [35]:
def get_geography_page(word):
    txt = None 
    try: 
        txt = wikipedia.WikipediaPage(word).section('Geography') 
    except:
        pass 
    return txt 

def get_summary_page(word):
    txt = None 
    try: 
        txt = wikipedia.summary(word)
    except:
        pass 
    return txt 

get_geography_page('Anhui')
# wikipedia.WikipediaPage('Anhui').section('Geography') 
get_summary_page('Anhui')

"The Hui people (Chinese: 回族; pinyin: Huízú; Wade–Giles: Hui2-tsu2, Xiao'erjing: خوذو, Dungan: Хуэйзў, Xuejzw) are an East Asian ethnoreligious group predominantly composed of Chinese-speaking adherents of Islam. They are distributed throughout China, mainly in the northwestern provinces and in the Zhongyuan region. According to the 2011 census, China is home to approximately 10.5 million Hui people. The 110,000 Dungan people of Kazakhstan and Kyrgyzstan are also considered part of the Hui ethnicity.\nThe Hui have a distinct connection with Islamic culture. For example, they follow Islamic dietary laws and reject the consumption of pork, the most commonly consumed meat in China, and have developed their own variation of Chinese cuisine. They also dress differently than the Han Chinese, some men wear white caps (taqiyah) and some women wear headscarves, as is the case in many Islamic cultures.\n\nThe Hui people are one of 56 ethnic groups recognized by China. The government defines the 

In [36]:
#attach wikipedia texts, using summary if no geographical information is found 
def get_wiki2(location_ls):
#initiaise txt with empty string 
    txt= None
    #loop over location list 
    for loc in location_ls: 
        #search similar words in wikipedia 
        similar_words = wikipedia.search(loc)   
        #first try getting geographical information looping over similar words 
        for similar in similar_words:
            txt = get_geography_page(similar)
            #return text if one of the similar words have geography section 
            if txt is not None:
#                 print('geography', similar)
                return txt  
            
    #if no geography section is present, get the summary  wikipedia page 
    for loc in location_ls:
        #search similar words in wikipedia 
        similar_words = wikipedia.search(loc) 
        for similar in similar_words:
            txt = get_summary_page(similar)
            if txt is not None: 
#                 print('summary', similar)
                return txt 
get_wiki2(['Anhui'])

'Anhui is topographically diverse. The north is part of the North China Plain while the north-central areas are part of the Huai River watershed. Both regions are flat, and densely populated. The land becomes more uneven further south, with the Dabie Mountains occupying much of southwestern Anhui and a series of hills and ranges cutting through southeastern Anhui, between which is the Yangtze River. The highest peak in Anhui is Lotus Peak, part of Huangshan in southeastern Anhui. It has an altitude of 1873 m.\nMajor rivers include the Huai River in the north and the Yangtze in the south. The largest lake is Lake Chaohu situated in the center of the province, with an area of about 800 km2 (310 sq mi). The southeastern part of the province near the Yangtze River has many lakes as well.\nAs with topography, the province differs in climate from north to south. The north is more temperate with more distinct seasons. January temperatures average at around −1 to 2 °C north of the Huai River, 

In [40]:
i = 0 
for index, row in grid_loc.iterrows():
    loc_ls = row['location']
#     print(loc_ls)
    txt = get_wiki2(loc_ls)
    grid_loc.loc[index, 'txt'] = txt 
    
    i +=1 
    #save output every 100 iterations 
    if i in range(1,10000,100):
        print(i)
        
    file = open('../../data/grid_loc_txt_new.pkl', 'wb')
    # dump information to that file
    pickle.dump(grid_loc, file )
    # close the file
    file.close()
    
    

1


/usr/local/lib/python3.9/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.9/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801


In [42]:
grid_loc

,grid_id,location,txt
0,"(-54, -72)",[Punta Arenas],"Located on the Brunswick Peninsula, Punta Aren..."
1,"(-49, -73)",[ O'Higgins],In pre-Quaternary times extensive Nothofagus f...
2,"(-49, -70)",[Santa Cruz],Santa Cruz is on the northern edge of Monterey...
3,"(-46, -73)","[Aysen region, Aisen del Gral. Carlos Ibañez d...",The Aysén del General Carlos Ibáñez del Campo ...
4,"(-46, 168)","[Invercargill, Mataura Borough, Invercargil, S...",Invercargill is the southernmost city in the C...
...,...,...,...
2822,"(65, 37)",[Eastern Maritime region],Indonesia is the largest country in Southeast ...
2823,"(66, -23)","[Sudavik, Flateyri]","The principal settlement is Ísafjörður, others..."
2824,"(66, 130)","[Sakha-Yakutia, Yakutia, Sakha Rep. provinces]",Borders:\ninternal: Chukotka Autonomous Okrug ...
2825,"(68, -135)",[Aklavik],Inuvik is located on the East Channel of the M...


In [43]:
#save to pickle 
file = open('../../data/grid_loc_txt_new.pkl', 'wb')
# dump information to that file
pickle.dump(grid_loc,file)

In [57]:
import pickle
# dump information to that file
grid_txt = pickle.load(open('../../data/grid_loc_txt_new.pkl', 'rb'))
# close the file
file.close()

In [58]:
grid_txt['txt'].isna().sum()

80

In [59]:
grid_txt

,grid_id,location,txt
0,"(-54, -72)",[Punta Arenas],"Located on the Brunswick Peninsula, Punta Aren..."
1,"(-49, -73)",[ O'Higgins],In pre-Quaternary times extensive Nothofagus f...
2,"(-49, -70)",[Santa Cruz],Santa Cruz is on the northern edge of Monterey...
3,"(-46, -73)","[Aysen region, Aisen del Gral. Carlos Ibañez d...",The Aysén del General Carlos Ibáñez del Campo ...
4,"(-46, 168)","[Invercargill, Mataura Borough, Invercargil, S...",Invercargill is the southernmost city in the C...
...,...,...,...
2822,"(65, 37)",[Eastern Maritime region],Indonesia is the largest country in Southeast ...
2823,"(66, -23)","[Sudavik, Flateyri]","The principal settlement is Ísafjörður, others..."
2824,"(66, 130)","[Sakha-Yakutia, Yakutia, Sakha Rep. provinces]",Borders:\ninternal: Chukotka Autonomous Okrug ...
2825,"(68, -135)",[Aklavik],Inuvik is located on the East Channel of the M...


## Next, construct 'LABEL' for training NLP 

In [49]:
#filter to flood only 
gdis_flood = gdis.loc[gdis['disastertype']=='flood']
flood_ct = gdis_flood.groupby('grid_id').agg({'disastertype':'count'}).rename(columns={'disastertype':'flood_ct'}).reset_index()
grid_txt_1 = pd.merge(grid_txt, flood_ct, on='grid_id', how='left').fillna(0)

In [60]:
#save to pickle 
file = open('../../data/grid_loc_txt_new.pkl', 'wb')
# dump information to that file
pickle.dump(grid_txt_1,file)

In [64]:
# import pickle
# # dump information to that file
df= pickle.load(open('../../data/grid_loc_txt_new.pkl', 'rb'))
# # close the file
# # file.close()

In [68]:
df

,grid_id,location,txt,flood_ct
0,"(-54, -72)",[Punta Arenas],"Located on the Brunswick Peninsula, Punta Aren...",1.0
1,"(-49, -73)",[ O'Higgins],In pre-Quaternary times extensive Nothofagus f...,1.0
2,"(-49, -70)",[Santa Cruz],Santa Cruz is on the northern edge of Monterey...,0.0
3,"(-46, -73)","[Aysen region, Aisen del Gral. Carlos Ibañez d...",The Aysén del General Carlos Ibáñez del Campo ...,0.0
4,"(-46, 168)","[Invercargill, Mataura Borough, Invercargil, S...",Invercargill is the southernmost city in the C...,4.0
...,...,...,...,...
2822,"(65, 37)",[Eastern Maritime region],Indonesia is the largest country in Southeast ...,1.0
2823,"(66, -23)","[Sudavik, Flateyri]","The principal settlement is Ísafjörður, others...",0.0
2824,"(66, 130)","[Sakha-Yakutia, Yakutia, Sakha Rep. provinces]",Borders:\ninternal: Chukotka Autonomous Okrug ...,2.0
2825,"(68, -135)",[Aklavik],Inuvik is located on the East Channel of the M...,1.0


In [74]:
!pip3 install pandas --upgrade

     |████████████████████████████████| 12.0 MB 1.3 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [75]:
df.to_csv('../../data/grid_loc_txt_new.csv')

In [50]:
# !pip3 install pandas==1.3.5

  Using cached pandas-1.3.5-cp39-cp39-macosx_10_9_x86_64.whl (11.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.2
    Uninstalling pandas-1.5.2:
      Successfully uninstalled pandas-1.5.2


In [52]:
# import pickle
# # dump information to that file
df= pickle.load(open('../../data/grid_loc_txt_new.pkl', 'rb'))
# # close the file
# # file.close()

In [62]:
# df

In [54]:
# pickle.HIGHEST_PROTOCOL = 4
# data.to_hdf('output/folder/path/to/file.hdf', 'df')

NameError: name 'data' is not defined

In [63]:
# #save to csv 
# df.to_csv('../data/grid_loc_txt.csv', index=False)